# 3. Building Compositional Space Complexes to Express Elaborate Design Problems

> ***(before running this code, please consult Quick Start to make sure everything is set up)***

> You will need the same dependencies as in [Tutorial No.2](02.AdditiveManufacturingPathPlanning.ipynb) (i.e., `pycalphad` and `pathfinding`), and additionally `igraph` in order to efficiently visualize the examples presented in this tutorial. **If you are running this in `nimplex`'s Codespaces, everything has been pre-installed for you.**

**In this tutorial, we will build upon `nimplex`'s graph representations and leverage our ability to combine ("stitch") them together without affecting the way we express the problem at hand and in an effortless fashion to dramatically speed up materials exploration.** In particular, we will show how to:

1. **Plot graphs** (as in mathematical objects, not any figures) using [`igraph`](https://igraph.org/), which can be done quite directly using `nimplex`'s point grid and neighbor lists `edges = [(i,n) for i in range(len(gridAtt)) for n in nList[i]]`

2. **Identify ordered subsystems in `nimplex` grids, like `A-B-C` and `C-A-B` within `A-B-C-D-E` and `D-C-G-F-A-H-B` to establish connectivity between them.** We will use this to combine 3 4-component systems (tetrahedra) by 2 3-component subsystems (triangles) to create a chain.

3. Identify all subspaces of a given order in low dimensional and high dimensional spaces (e.g., all quantized compositions of any 3 things out of N) and combine ("stitch") them together to form a **simplex graphs that intersect themselves in 3D because of high dimensionality, yet still have the same (graph) structure**.

4. **Construct a graph complex to explore all ternary combinations of `["Ti50Zr50", "Hf95Ti5", "NbTaWHf", "Mo80Nb10W10", "TiTa2", "Nb96Mo3W1", "Zr49 Hf1 Mo50"]` (7 alloys)** under equilibrium phase constraint (relatively expensive to compute) and then explore the space with an additional low-cost screenig constraint (RMSAD - alloy strenght surrogate) that could also be an ML surrogate.

In [1]:
import nimplex
from utils import stitching
from IPython.display import clear_output
from itertools import combinations

In [2]:
import igraph as ig
import plotly.graph_objs as go
import numpy as np
import random
random.seed(123)

## Plotting 4-Component Simplex Graph with `igraph`

In [96]:
dim = 4
ndiv = 6
gridAtt, nList = nimplex.simplex_graph_py(dim, ndiv)

In [97]:
edges = []
for i in range(len(gridAtt)):
    for n in nList[i]:
        edges.append((i,n))

In [98]:
def plotGraph(edges: list):
    # Lets generate some seed positions for the nodes
    G = ig.Graph(edges)
    layout = G.layout_kamada_kawai(dim=3)
    layout_array = np.array(layout.coords)
    # Create edge traces
    edge_traces = []
    for edge in G.es:
        start, end = edge.tuple
        x0, y0, z0 = layout_array[start]
        x1, y1, z1 = layout_array[end]
        width = 5
        edge_trace = go.Scatter3d(x=[x0, x1], y=[y0, y1], z=[z0, z1],
                                line=dict(width=width, color='lightgray'),
                                opacity=0.2,
                                hoverinfo='none', mode='lines')
        edge_traces.append(edge_trace)

    node_trace = go.Scatter3d(x=layout_array[:, 0], y=layout_array[:, 1], z=layout_array[:, 2],
                            mode='markers',
                            marker=dict(size=3, 
                                        opacity=0.5,
                                        line=None,
                                        color='blue'),
                            
                            text=[f"Node {i}" for i in range(len(gridAtt))],
                            hoverinfo='text')

    # Create figure
    fig = go.Figure(data=edge_traces + [node_trace])

    # Update layout
    fig.update_layout(
        title='3D Graph with Feasibility Types',
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
        ),
        showlegend=False,
        margin=dict(l=0, r=0, b=0, t=0),
        width=800
    )

    fig.show()

In [99]:
#plotGraph(edges)
clear_output()

## Combining ("Stitching") 3 4-Component Systems into A Complex

In [100]:
sys1 = ["A", "B", "C", "D"]
sys2 = ["B", "C", "D", "E"]
sys3 = ["B", "D", "E", "F"]

In [101]:
offset = len(gridAtt)
for i in range(len(gridAtt)):
    for n in nList[i]:
        edges.append((i+offset,n+offset))
for i in range(len(gridAtt)):
    for n in nList[i]:
        edges.append((i+offset*2,n+offset*2))

In [102]:
#plotGraph(edges)
clear_output()

In [103]:
stitchingPoints = stitching.findStitchingPoints_py(dim, ndiv, components=["A", "B", "C", "D"])
print(stitchingPoints["A-B"])

[83, 82, 79, 73, 63, 48, 27]


In [104]:
stitchingPoints1 = stitching.findStitchingPoints_py(dim, ndiv, components=sys1, offset=0)
stitchingPoints2 = stitching.findStitchingPoints_py(dim, ndiv, components=sys2, offset=len(gridAtt))
stitchingPoints3 = stitching.findStitchingPoints_py(dim, ndiv, components=sys3, offset=len(gridAtt)*2)

In [105]:
for i, j in zip(stitchingPoints1["B-C-D"], stitchingPoints2["B-C-D"]):
    edges.append((i, j))

In [106]:
for i, j in zip(stitchingPoints2["B-D-E"], stitchingPoints3["B-D-E"]):
    edges.append((i, j))

In [107]:
#plotGraph(edges)
clear_output()

## Combining ("Stitching") 9 3-Component Systems into A Complex
### Disregard this section for now Adam

In [3]:
dim = 3
ndiv = 6
nsys = 9
gridAtt, nList = nimplex.simplex_graph_py(dim, ndiv)
edges = []
for i in range(len(gridAtt)):
    for n in nList[i]:
        edges.append((i,n))

In [4]:
len(gridAtt)

28

In [5]:
offset = len(gridAtt)
for sys in range(1,nsys+1):
    for i in range(len(gridAtt)):
        for n in nList[i]:
            edges.append((i+offset*sys,n+offset*sys))

In [6]:
#plotGraph(edges)
clear_output()

In [7]:
from pycalphad import Database
dbCrNiV = Database("ammap/databases/Co-Cr-Fe-Ni-V_choi2019.TDB")
dbCrFeTi = Database("ammap/databases/Cr-Fe-Ti_wang2017.tdb")
dbCrFeNi = Database("ammap/databases/Cr-Fe-Ni_miettinen1999.tdb")
dbCrNiTi= Database("ammap/databases/Cr-Ni-Ti_huang2018.tdb")
dbCrTiV= Database("ammap/databases/Cr-Ti-V_ghosh2002.tdb")
dbFeNiTi= Database("ammap/databases/Fe-Ni-Ti_dekeyzer2009.tdb")
dbFeNiV= Database("ammap/databases/Fe-Ni-V_zhao2014.tdb")
dbFeTiV= Database("ammap/databases/Fe-Ti-V_guo2012.TDB")
dbNiTiV= Database("ammap/databases/Ni-Ti-V_zou2018.tdb")
CrNiV = ['Cr', 'Ni', 'V']
CrFeTi = ['Cr', 'Fe', 'Ti']
CrFeNi = ['Cr', 'Fe', 'Ni']
CrNiTi = ['Cr', 'Ni', 'Ti']
CrTiV = ['Cr', 'Ti', 'V']
FeNiTi = ['Fe', 'Ni', 'Ti']
FeNiV = ['Fe', 'Ni', 'V']
FeTiV = ['Fe', 'Ti', 'V']
NiTiV = ['Ni', 'Ti', 'V']


/opt/conda/lib/python3.11/site-packages/pycalphad/io/tdb.py:293: UserWarning: The type definition character `&` in `TYPE_DEFINITION & GES A_P_D A2_BCC MAGNETIC -1.0 4.00000E-01 ` is not used by any phase.
  warnings.warn(f"The type definition character `{typechar}` in `TYPE_DEFINITION {typechar} {line}` is not used by any phase.")


In [113]:
stitchingPointsCrNiV = stitching.findStitchingPoints_py(dim, ndiv, components=CrNiV, offset=0)
stitchingPointsCrFeTi = stitching.findStitchingPoints_py(dim, ndiv, components=CrFeTi, offset=len(gridAtt))
stitchingPointsCrFeNi = stitching.findStitchingPoints_py(dim, ndiv, components=CrFeNi, offset=len(gridAtt)*2)
stitchingPointsCrNiTi = stitching.findStitchingPoints_py(dim, ndiv, components=CrNiTi, offset=len(gridAtt)*3)
stitchingPointsCrTiV = stitching.findStitchingPoints_py(dim, ndiv, components=CrTiV, offset=len(gridAtt)*4)
stitchingPointsFeNiTi = stitching.findStitchingPoints_py(dim, ndiv, components=FeNiTi, offset=len(gridAtt)*5)
stitchingPointsFeNiV = stitching.findStitchingPoints_py(dim, ndiv, components=FeNiV, offset=len(gridAtt)*6)
stitchingPointsFeTiV = stitching.findStitchingPoints_py(dim, ndiv, components=FeTiV, offset=len(gridAtt)*7)
stitchingPointsNiTiV = stitching.findStitchingPoints_py(dim, ndiv, components=NiTiV, offset=len(gridAtt)*8)

## Finding and Combining 3-Component Subspaces in 4-Component Space

In [114]:
from itertools import combinations
elements = ["A", "B", "C", "D"]
ternaries = list(combinations(elements, 3))
print(ternaries)

[('A', 'B', 'C'), ('A', 'B', 'D'), ('A', 'C', 'D'), ('B', 'C', 'D')]


In [115]:
gridAtt, nList = nimplex.simplex_graph_py(3,12)

In [116]:
edges = []
# Ternaries
for i, ternary in enumerate(ternaries):
    offset = i*len(gridAtt)
    for i in range(len(gridAtt)):
        for n in nList[i]:
            edges.append((i+offset,n+offset))

In [ ]:
#plotGraph(edges)
clear_output()

In [118]:
stitchingBinaries = {}

for i, combo1 in enumerate(ternaries):
    for j, combo2 in enumerate(ternaries[i+1:], start=i+1):
        common = set(combo1) & set(combo2)
        if len(common) == 2:
            overlap = tuple(sorted(common))
            if overlap not in stitchingBinaries:
                stitchingBinaries[overlap] = []
            stitchingBinaries[overlap].append((i, j))

for overlap, pairs in stitchingBinaries.items():
    print(f"{overlap}: occurs between ternary {pairs}")

('A', 'B'): occurs between ternary [(0, 1)]
('A', 'C'): occurs between ternary [(0, 2)]
('B', 'C'): occurs between ternary [(0, 3)]
('A', 'D'): occurs between ternary [(1, 2)]
('B', 'D'): occurs between ternary [(1, 3)]
('C', 'D'): occurs between ternary [(2, 3)]


In [119]:
for stitchingBinary, ternaryPair in stitchingBinaries.items():
    ternary1, ternary2 = ternaryPair[0][0], ternaryPair[0][1]
    stitching1 = stitching.findStitchingPoints_py(
        3, 12, 
        components=ternaries[ternary1],
        offset=ternary1*len(gridAtt)
        )["-".join(stitchingBinary)]
    stitching2 = stitching.findStitchingPoints_py(
        3, 12, 
        components=ternaries[ternary2],
        offset=ternary2*len(gridAtt)
        )["-".join(stitchingBinary)]
    print(f"Stitching {ternary1} and {ternary2} at {stitchingBinary} from {stitching1} to {stitching2}")
    for i, j in zip(stitching1, stitching2):
        edges.append((i, j))

Stitching 0 and 1 at ('A', 'B') from [90, 89, 87, 84, 80, 75, 69, 62, 54, 45, 35, 24, 12] to [181, 180, 178, 175, 171, 166, 160, 153, 145, 136, 126, 115, 103]
Stitching 0 and 2 at ('A', 'C') from [90, 88, 85, 81, 76, 70, 63, 55, 46, 36, 25, 13, 0] to [272, 271, 269, 266, 262, 257, 251, 244, 236, 227, 217, 206, 194]
Stitching 0 and 3 at ('B', 'C') from [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0] to [363, 362, 360, 357, 353, 348, 342, 335, 327, 318, 308, 297, 285]
Stitching 1 and 2 at ('A', 'D') from [181, 179, 176, 172, 167, 161, 154, 146, 137, 127, 116, 104, 91] to [272, 270, 267, 263, 258, 252, 245, 237, 228, 218, 207, 195, 182]
Stitching 1 and 3 at ('B', 'D') from [103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91] to [363, 361, 358, 354, 349, 343, 336, 328, 319, 309, 298, 286, 273]
Stitching 2 and 3 at ('C', 'D') from [194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182] to [285, 284, 283, 282, 281, 280, 279, 278, 277, 276, 275, 274, 273]


In [120]:
#plotGraph(edges)
clear_output()

## In Higher Dimensional Cases (Stretched 2D Spaces Crossing Each Other in 3D)

In [121]:
from itertools import combinations
elements = ["A", "B", "C", "D", "E"]
ternaries = list(combinations(elements, 3))
print(ternaries)

[('A', 'B', 'C'), ('A', 'B', 'D'), ('A', 'B', 'E'), ('A', 'C', 'D'), ('A', 'C', 'E'), ('A', 'D', 'E'), ('B', 'C', 'D'), ('B', 'C', 'E'), ('B', 'D', 'E'), ('C', 'D', 'E')]


In [122]:
gridAtt, nList = nimplex.simplex_graph_py(3,12)

In [123]:
edges = []
# Ternaries
for i, ternary in enumerate(ternaries):
    offset = i*len(gridAtt)
    for i in range(len(gridAtt)):
        for n in nList[i]:
            edges.append((i+offset,n+offset))


In [124]:
#plotGraph(edges)
clear_output()

In [125]:
stitchingBinaries = {}

for i, combo1 in enumerate(ternaries):
    for j, combo2 in enumerate(ternaries[i+1:], start=i+1):
        common = set(combo1) & set(combo2)
        if len(common) == 2:
            overlap = tuple(sorted(common))
            if overlap not in stitchingBinaries:
                stitchingBinaries[overlap] = []
            stitchingBinaries[overlap].append((i, j))

for overlap, pairs in stitchingBinaries.items():
    print(f"{overlap}: occurs between ternary {pairs}")

('A', 'B'): occurs between ternary [(0, 1), (0, 2), (1, 2)]
('A', 'C'): occurs between ternary [(0, 3), (0, 4), (3, 4)]
('B', 'C'): occurs between ternary [(0, 6), (0, 7), (6, 7)]
('A', 'D'): occurs between ternary [(1, 3), (1, 5), (3, 5)]
('B', 'D'): occurs between ternary [(1, 6), (1, 8), (6, 8)]
('A', 'E'): occurs between ternary [(2, 4), (2, 5), (4, 5)]
('B', 'E'): occurs between ternary [(2, 7), (2, 8), (7, 8)]
('C', 'D'): occurs between ternary [(3, 6), (3, 9), (6, 9)]
('C', 'E'): occurs between ternary [(4, 7), (4, 9), (7, 9)]
('D', 'E'): occurs between ternary [(5, 8), (5, 9), (8, 9)]


In [126]:
for stitchingBinary, ternaryPairList in stitchingBinaries.items():
    for ternaryPair in ternaryPairList:
        ternary1, ternary2 = ternaryPair[0], ternaryPair[1]
        stitching1 = stitching.findStitchingPoints_py(
            3, 12, 
            components=ternaries[ternary1],
            offset=ternary1*len(gridAtt)
            )["-".join(stitchingBinary)]
        stitching2 = stitching.findStitchingPoints_py(
            3, 12, 
            components=ternaries[ternary2],
            offset=ternary2*len(gridAtt)
            )["-".join(stitchingBinary)]
        print(f"Stitching {ternary1} and {ternary2} at {stitchingBinary} from {stitching1} to {stitching2}")
        for i, j in zip(stitching1, stitching2):
            edges.append((i, j))

Stitching 0 and 1 at ('A', 'B') from [90, 89, 87, 84, 80, 75, 69, 62, 54, 45, 35, 24, 12] to [181, 180, 178, 175, 171, 166, 160, 153, 145, 136, 126, 115, 103]
Stitching 0 and 2 at ('A', 'B') from [90, 89, 87, 84, 80, 75, 69, 62, 54, 45, 35, 24, 12] to [272, 271, 269, 266, 262, 257, 251, 244, 236, 227, 217, 206, 194]
Stitching 1 and 2 at ('A', 'B') from [181, 180, 178, 175, 171, 166, 160, 153, 145, 136, 126, 115, 103] to [272, 271, 269, 266, 262, 257, 251, 244, 236, 227, 217, 206, 194]
Stitching 0 and 3 at ('A', 'C') from [90, 88, 85, 81, 76, 70, 63, 55, 46, 36, 25, 13, 0] to [363, 362, 360, 357, 353, 348, 342, 335, 327, 318, 308, 297, 285]
Stitching 0 and 4 at ('A', 'C') from [90, 88, 85, 81, 76, 70, 63, 55, 46, 36, 25, 13, 0] to [454, 453, 451, 448, 444, 439, 433, 426, 418, 409, 399, 388, 376]
Stitching 3 and 4 at ('A', 'C') from [363, 362, 360, 357, 353, 348, 342, 335, 327, 318, 308, 297, 285] to [454, 453, 451, 448, 444, 439, 433, 426, 418, 409, 399, 388, 376]
Stitching 0 and 6 at (

In [127]:
#plotGraph(edges)
clear_output()

## Exploring Complex Formed by All 3-Component Alloy Subsystems of 5-Component Space (w. Infeasibility Gliding) #THIS ONE ADAM

In [8]:
elementalSpaceComponents = ["Cr","Fe","Ni","Ti","V"]
#attainableSpaceComponents = ["SS304L", "Ni", "Cr", "V", "Ti"]
attainableSpaceComponents = ["Cr", "Fe", "Ni", "Ti", "V"]
attainableSpaceComponentPositions = [
    #"Cr","Fe","Ni","Ti","V"
    #[0.1993865031, 0.7044989775,0.096114519430,0,0],
    [1,0,0,0,0],
    [0,1,0,0,0],
    [0,0,1,0,0],
    [0,0,0,1,0],
    [0,0,0,0,1]
]    
ternaries = list(combinations(attainableSpaceComponents, 3))
ternaries_CompPos = list(combinations(attainableSpaceComponentPositions, 3))
ndiv = 8
gridAtt, nList = nimplex.simplex_graph_py(3, ndiv)

for tern, terncp in zip(ternaries, ternaries_CompPos):
    print(f"{str(tern):<40} -> {terncp}")

('Cr', 'Fe', 'Ni')                       -> ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0])
('Cr', 'Fe', 'Ti')                       -> ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0])
('Cr', 'Fe', 'V')                        -> ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1])
('Cr', 'Ni', 'Ti')                       -> ([1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0])
('Cr', 'Ni', 'V')                        -> ([1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1])
('Cr', 'Ti', 'V')                        -> ([1, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1])
('Fe', 'Ni', 'Ti')                       -> ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0])
('Fe', 'Ni', 'V')                        -> ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1])
('Fe', 'Ti', 'V')                        -> ([0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1])
('Ni', 'Ti', 'V')                        -> ([0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1])


In [9]:
# Edges list for graph plotting and path finding purposes
edges = []
# Connectivity list within each subsystem
graphN = [[] for i in range(len(gridAtt * len(ternaries)))]
# Connectivity list between subsystems
graphNS = [[] for i in range(len(graphN))]
compositions = []
compositions_with_id = []  # List to store compositions with their identifiers
ternaries_with_id = []  # New list to store ternaries_CompPos with their identifiers

# Iterate over ternaries
for i, terncp in enumerate(ternaries_CompPos):
    ternaries_with_id.append((terncp, i))  # Add terncp and its id to the new list
    
    offset = i*len(gridAtt)
    for j in range(len(gridAtt)):
        for n in nList[j]:
            edges.append((j+offset,n+offset))
            graphN[j+offset].append(n+offset)
    gridAttTemp, gridElTemp = nimplex.embeddedpair_simplex_grid_fractional_py(terncp, ndiv)
    compositions += gridElTemp
    
    # Attach identifier to each composition
    compositions_with_id.extend([(comp, i) for comp in gridElTemp])

In [10]:
# Example: Access ternaries_CompPos with their identifiers
for terncp, ternary_id in ternaries_with_id:
    print(f"Ternary composition position {terncp} has id {ternary_id}")

Ternary composition position ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0]) has id 0
Ternary composition position ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0]) has id 1
Ternary composition position ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1]) has id 2
Ternary composition position ([1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0]) has id 3
Ternary composition position ([1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1]) has id 4
Ternary composition position ([1, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]) has id 5
Ternary composition position ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0]) has id 6
Ternary composition position ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1]) has id 7
Ternary composition position ([0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]) has id 8
Ternary composition position ([0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]) has id 9


In [11]:
mapping = {}
for ternary, id in ternaries_with_id:
    composition_key = ''.join(elementalSpaceComponents[i] for i in range(len(elementalSpaceComponents)) if any(ternary[j][i] for j in range(len(ternary))))
    mapping[composition_key] = id

In [12]:
mapping = {}
for ternary, id in ternaries_with_id:
    composition_key = ''.join(elementalSpaceComponents[i] for i in range(len(elementalSpaceComponents)) if any(ternary[j][i] for j in range(len(ternary))))
    individual_elements = [elementalSpaceComponents[i] for i in range(len(elementalSpaceComponents)) if any(ternary[j][i] for j in range(len(ternary)))]
    mapping[composition_key] = {
        'id': id,
        'elements': individual_elements
    }

# Print the mapping to see the result
for key, value in mapping.items():
    print(f"Combination: {key}, ID: {value['id']}, Elements: {value['elements']}")

Combination: CrFeNi, ID: 0, Elements: ['Cr', 'Fe', 'Ni']
Combination: CrFeTi, ID: 1, Elements: ['Cr', 'Fe', 'Ti']
Combination: CrFeV, ID: 2, Elements: ['Cr', 'Fe', 'V']
Combination: CrNiTi, ID: 3, Elements: ['Cr', 'Ni', 'Ti']
Combination: CrNiV, ID: 4, Elements: ['Cr', 'Ni', 'V']
Combination: CrTiV, ID: 5, Elements: ['Cr', 'Ti', 'V']
Combination: FeNiTi, ID: 6, Elements: ['Fe', 'Ni', 'Ti']
Combination: FeNiV, ID: 7, Elements: ['Fe', 'Ni', 'V']
Combination: FeTiV, ID: 8, Elements: ['Fe', 'Ti', 'V']
Combination: NiTiV, ID: 9, Elements: ['Ni', 'Ti', 'V']


In [13]:
mapping

{'CrFeNi': {'id': 0, 'elements': ['Cr', 'Fe', 'Ni']},
 'CrFeTi': {'id': 1, 'elements': ['Cr', 'Fe', 'Ti']},
 'CrFeV': {'id': 2, 'elements': ['Cr', 'Fe', 'V']},
 'CrNiTi': {'id': 3, 'elements': ['Cr', 'Ni', 'Ti']},
 'CrNiV': {'id': 4, 'elements': ['Cr', 'Ni', 'V']},
 'CrTiV': {'id': 5, 'elements': ['Cr', 'Ti', 'V']},
 'FeNiTi': {'id': 6, 'elements': ['Fe', 'Ni', 'Ti']},
 'FeNiV': {'id': 7, 'elements': ['Fe', 'Ni', 'V']},
 'FeTiV': {'id': 8, 'elements': ['Fe', 'Ti', 'V']},
 'NiTiV': {'id': 9, 'elements': ['Ni', 'Ti', 'V']}}

In [14]:
# def find_duplicates(lst):
#     duplicates_counts = {}
#     for item in lst:
#         # Use the string representation for complex/unhashable types
#         item_str = str(item)
#         if item_str in duplicates_counts:
#             duplicates_counts[item_str] += 1
#         else:
#             duplicates_counts[item_str] = 1

#     # Filter out items that only appear once
#     duplicates = {item: count for item, count in duplicates_counts.items() if count > 1}
    
#     if not duplicates:
#         return "No duplicates found in the list."
#     else:
#         result = "Duplicates found:"
#         for item, count in duplicates.items():
#             result += f"{item} appears {count} times"
#         return result.strip()

In [15]:
# find_duplicates(compositions)

In [16]:
stitchingBinaries = {}

for i, combo1 in enumerate(ternaries):
    for j, combo2 in enumerate(ternaries[i+1:], start=i+1):
        common = set(combo1) & set(combo2)
        if len(common) == 2:
            overlap = tuple(sorted(common))
            if overlap not in stitchingBinaries:
                stitchingBinaries[overlap] = []
            stitchingBinaries[overlap].append((i, j))

for overlap, pairs in stitchingBinaries.items():
    print(f"{overlap}: occurs between ternary {pairs}")

('Cr', 'Fe'): occurs between ternary [(0, 1), (0, 2), (1, 2)]
('Cr', 'Ni'): occurs between ternary [(0, 3), (0, 4), (3, 4)]
('Fe', 'Ni'): occurs between ternary [(0, 6), (0, 7), (6, 7)]
('Cr', 'Ti'): occurs between ternary [(1, 3), (1, 5), (3, 5)]
('Fe', 'Ti'): occurs between ternary [(1, 6), (1, 8), (6, 8)]
('Cr', 'V'): occurs between ternary [(2, 4), (2, 5), (4, 5)]
('Fe', 'V'): occurs between ternary [(2, 7), (2, 8), (7, 8)]
('Ni', 'Ti'): occurs between ternary [(3, 6), (3, 9), (6, 9)]
('Ni', 'V'): occurs between ternary [(4, 7), (4, 9), (7, 9)]
('Ti', 'V'): occurs between ternary [(5, 8), (5, 9), (8, 9)]


In [17]:
for stitchingBinary, ternaryPairList in stitchingBinaries.items():
    for ternaryPair in ternaryPairList:
        ternary1, ternary2 = ternaryPair[0], ternaryPair[1]
        stitching1 = stitching.findStitchingPoints_py(
            3, ndiv, 
            components=ternaries[ternary1],
            offset=ternary1*len(gridAtt)
            )["-".join(stitchingBinary)]
        stitching2 = stitching.findStitchingPoints_py(
            3, ndiv, 
            components=ternaries[ternary2],
            offset=ternary2*len(gridAtt)
            )["-".join(stitchingBinary)]
        print(f"Stitching {ternary1} and {ternary2} at {stitchingBinary} from {stitching1} to {stitching2}")
        for i, j in zip(stitching1, stitching2):
            edges.append((i, j))
            graphNS[i].append(j)

Stitching 0 and 1 at ('Cr', 'Fe') from [44, 43, 41, 38, 34, 29, 23, 16, 8] to [89, 88, 86, 83, 79, 74, 68, 61, 53]
Stitching 0 and 2 at ('Cr', 'Fe') from [44, 43, 41, 38, 34, 29, 23, 16, 8] to [134, 133, 131, 128, 124, 119, 113, 106, 98]
Stitching 1 and 2 at ('Cr', 'Fe') from [89, 88, 86, 83, 79, 74, 68, 61, 53] to [134, 133, 131, 128, 124, 119, 113, 106, 98]
Stitching 0 and 3 at ('Cr', 'Ni') from [44, 42, 39, 35, 30, 24, 17, 9, 0] to [179, 178, 176, 173, 169, 164, 158, 151, 143]
Stitching 0 and 4 at ('Cr', 'Ni') from [44, 42, 39, 35, 30, 24, 17, 9, 0] to [224, 223, 221, 218, 214, 209, 203, 196, 188]
Stitching 3 and 4 at ('Cr', 'Ni') from [179, 178, 176, 173, 169, 164, 158, 151, 143] to [224, 223, 221, 218, 214, 209, 203, 196, 188]
Stitching 0 and 6 at ('Fe', 'Ni') from [8, 7, 6, 5, 4, 3, 2, 1, 0] to [314, 313, 311, 308, 304, 299, 293, 286, 278]
Stitching 0 and 7 at ('Fe', 'Ni') from [8, 7, 6, 5, 4, 3, 2, 1, 0] to [359, 358, 356, 353, 349, 344, 338, 331, 323]
Stitching 6 and 7 at ('Fe'

In [18]:
from pycalphad import Database

# List of database file paths
db_files = [
    "ammap/databases/Co-Cr-Fe-Ni-V_choi2019.TDB",
    "ammap/databases/Cr-Fe-Ti_wang2017.tdb",
    "ammap/databases/Cr-Fe-Ni_miettinen1999.tdb",
    "ammap/databases/Cr-Ni-Ti_huang2018.tdb",
    "ammap/databases/Cr-Ti-V_ghosh2002.tdb",
    "ammap/databases/Fe-Ni-Ti_dekeyzer2009.tdb",
    "ammap/databases/Fe-Ni-V_zhao2014.tdb",
    "ammap/databases/Fe-Ti-V_guo2012.TDB",
    "ammap/databases/Ni-Ti-V_zou2018.tdb"
]

# Dictionary to store unique phases for each database
unique_phases = {}

# Iterate through each database file
for db_file in db_files:
    dbf = Database(db_file)
    phases = list(set(dbf.phases.keys()))
    unique_phases[db_file] = phases

# Print unique phases for each database
for db_file, phases in unique_phases.items():
    print(f"Unique phases for {db_file}: {phases}")

Unique phases for ammap/databases/Co-Cr-Fe-Ni-V_choi2019.TDB: ['B2_BCC', 'BCC_A2', 'NI2V', 'LIQUID', 'FCC_A1', 'SIGMA', 'HIGH_SIGMA', 'COV3', 'NI2V7', 'L12_FCC', 'M3V', 'HCP_A3']
Unique phases for ammap/databases/Cr-Fe-Ti_wang2017.tdb: ['BCC_A2', 'LIQUID', 'C36', 'FCC_A1', 'SIGMA', 'C15', 'C14', 'TI5CR7FE17', 'BCC_B2', 'HCP_A3']
Unique phases for ammap/databases/Cr-Fe-Ni_miettinen1999.tdb: ['BCC_A2', 'LIQUID', 'FCC_A1', 'SIGMA', 'HCP_A3']
Unique phases for ammap/databases/Cr-Ni-Ti_huang2018.tdb: ['BCC_A2', 'LIQUID', 'HCP_A3', 'FCC_A1', 'NITI', 'LAVES_C14', 'LAVES_C36', 'NITI2', 'NI3TI', 'LAVES_C15']
Unique phases for ammap/databases/Cr-Ti-V_ghosh2002.tdb: ['BCC_A2', 'LIQUID', 'LAVES_C14', 'LAVES_C36', 'LAVES_C15', 'HCP_A3']
Unique phases for ammap/databases/Fe-Ni-Ti_dekeyzer2009.tdb: ['BCC2', 'LIQUID', 'A3', 'NI3TI', 'A2', 'C14', 'FCC4', 'NITI2', 'A1']
Unique phases for ammap/databases/Fe-Ni-V_zhao2014.tdb: ['BCC_A2', 'NI2V', 'LIQUID', 'CBCC_A12', 'HCP_A3', 'FCC_A1', 'A15_NI2V7', 'SIGM

In [139]:

dbf = Database("CrHfMoNbTaTiVWZr_9element_Feb2023.tdb")
phases = list(set(dbf.phases.keys()))
print(elementalSpaceComponents)
print(f'Loaded TDB file with phases considered: {phases}')
from myPycalphadCallable import equilibrium_callable

['Cr', 'Fe', 'Ni', 'Ti', 'V']
Loaded TDB file with phases considered: ['HCP_A3', 'LAVES_C36', 'LAVES_C15', 'LIQUID', 'BCC_A2', 'LAVES_C14', 'FCC_A1']


In [140]:
import os
import importlib

# Directory containing the equilibrium files
directory = "ammap/callables/multi_system_equilibrium_and_scheil"

# Get all files starting with "equilibrium"
equilibrium_files = [f for f in os.listdir(directory) if f.startswith("equilibrium") and f.endswith(".py")]

# Dictionary to store imported callables with unique names
equilibrium_callables = {}

# Import each equilibrium file and store the callable with a unique name
for file in equilibrium_files:
    module_name = file[:-3]  # Remove the .py extension
    module_path = f"ammap.callables.multi_system_equilibrium_and_scheil.{module_name}"
    module = importlib.import_module(module_path)
    callable_name = f"{module_name}"
    equilibrium_callables[callable_name] = getattr(module, "equilibrium_callable")

# Print the imported callables
for name, func in equilibrium_callables.items():
    print(f"Imported {name}: {func}")

Imported equilibrium_callable_CrFeTi_08bbfb9a: <function equilibrium_callable at 0x7f09782125c0>
Imported equilibrium_callable_CrTiV_ed4c332b: <function equilibrium_callable at 0x7f09782113a0>
Imported equilibrium_callable_FeTiV_fa95b3ee: <function equilibrium_callable at 0x7f0979aad580>
Imported equilibrium_callable_NiCrFe_f434a6d9: <function equilibrium_callable at 0x7f0977426520>
Imported equilibrium_callable_CrFeV_ab1edb07: <function equilibrium_callable at 0x7f097748b1a0>
Imported equilibrium_callable_FeNiV_b9b0384d: <function equilibrium_callable at 0x7f0977567c40>
Imported equilibrium_callable_FeNiTi_06a49695: <function equilibrium_callable at 0x7f09775668e0>
Imported equilibrium_callable_CrNiTi_a9f6f2ff: <function equilibrium_callable at 0x7f0977583ba0>
Imported equilibrium_callable_NiTiV_1d83c99c: <function equilibrium_callable at 0x7f09769327a0>
Imported equilibrium_callable_NiCrV_b7aba9ab: <function equilibrium_callable at 0x7f09779e2ac0>


In [141]:
def process_filename(filename):
    parts = filename.split('_')
    if len(parts) >= 4:
        middle_part = parts[2]
        if middle_part in element_mapping:
            return middle_part, element_mapping[middle_part]
    return None, None

In [142]:
element_mapping=mapping

In [143]:
# import os

# # Assuming you have already created the mapping as shown in the previous example
# # mapping = {...}  # Your mapping dictionary

# def process_filename(filename, mapping):
#     parts = filename.split('_')
#     if len(parts) >= 4:
#         middle_part = parts[2]
#         # First, try direct matching
#         if middle_part in mapping:
#             return middle_part, mapping[middle_part]['id'], mapping[middle_part]['elements']
        
#         # If direct matching fails, try matching by elements
#         middle_elements = set(middle_part[i:i+2] for i in range(0, len(middle_part), 2))
#         for key, value in mapping.items():
#             if set(value['elements']) == middle_elements:
#                 return middle_part, value['id'], value['elements']
    
#     return None, None, None

# # Specify the directory path
# directory = "ammap/callables/multi_system_equilibrium_and_scheil"

# for filename in os.listdir(directory):
#     if filename.startswith("equilibrium"):
#         full_path = os.path.join(directory, filename)
#         if os.path.isfile(full_path):
#             middle_part, mapping_id, elements = process_filename(filename, mapping)
#             if middle_part and mapping_id is not None:
#                 print(f"File: {filename}")
#                 print(f"The mapping number for {middle_part} is {mapping_id}")
#                 print(f"Elements: {elements}")
#                 print("---")
#             else:
#                 print(f"No mapping found for file: {filename}")
#                 print("---")

In [144]:
def process_key(key):
    parts = key.split('_')
    if len(parts) >= 3:
        middle_part = parts[2]
        if middle_part in mapping:
            return middle_part, mapping[middle_part]['id'], mapping[middle_part]['elements']
        
        # If direct matching fails, try matching by elements
        middle_elements = set(middle_part[i:i+2] for i in range(0, len(middle_part), 2))
        for map_key, value in mapping.items():
            if set(value['elements']) == middle_elements:
                return middle_part, value['id'], value['elements']
    
    return None, None, None

# Process each key in the equilibrium_callables dictionary
id_to_callable = {}
for key in equilibrium_callables:
    middle_part, mapping_id, elements = process_key(key)
    if middle_part and mapping_id is not None:
        print(f"Key: {key}")
        print(f"The mapping number for {middle_part} is {mapping_id}")
        print(f"Elements: {elements}")
        print("---")
        id_to_callable[mapping_id]=key
    else:
        print(f"No mapping found for key: {key}")
        print("---")

Key: equilibrium_callable_CrFeTi_08bbfb9a
The mapping number for CrFeTi is 1
Elements: ['Cr', 'Fe', 'Ti']
---
Key: equilibrium_callable_CrTiV_ed4c332b
The mapping number for CrTiV is 5
Elements: ['Cr', 'Ti', 'V']
---
Key: equilibrium_callable_FeTiV_fa95b3ee
The mapping number for FeTiV is 8
Elements: ['Fe', 'Ti', 'V']
---
Key: equilibrium_callable_NiCrFe_f434a6d9
The mapping number for NiCrFe is 0
Elements: ['Cr', 'Fe', 'Ni']
---
Key: equilibrium_callable_CrFeV_ab1edb07
The mapping number for CrFeV is 2
Elements: ['Cr', 'Fe', 'V']
---
Key: equilibrium_callable_FeNiV_b9b0384d
The mapping number for FeNiV is 7
Elements: ['Fe', 'Ni', 'V']
---
Key: equilibrium_callable_FeNiTi_06a49695
The mapping number for FeNiTi is 6
Elements: ['Fe', 'Ni', 'Ti']
---
Key: equilibrium_callable_CrNiTi_a9f6f2ff
The mapping number for CrNiTi is 3
Elements: ['Cr', 'Ni', 'Ti']
---
Key: equilibrium_callable_NiTiV_1d83c99c
The mapping number for NiTiV is 9
Elements: ['Ni', 'Ti', 'V']
---
Key: equilibrium_callable

In [145]:
id_to_callable

{1: 'equilibrium_callable_CrFeTi_08bbfb9a',
 5: 'equilibrium_callable_CrTiV_ed4c332b',
 8: 'equilibrium_callable_FeTiV_fa95b3ee',
 0: 'equilibrium_callable_NiCrFe_f434a6d9',
 2: 'equilibrium_callable_CrFeV_ab1edb07',
 7: 'equilibrium_callable_FeNiV_b9b0384d',
 6: 'equilibrium_callable_FeNiTi_06a49695',
 3: 'equilibrium_callable_CrNiTi_a9f6f2ff',
 9: 'equilibrium_callable_NiTiV_1d83c99c',
 4: 'equilibrium_callable_NiCrV_b7aba9ab'}

In [146]:
compositions_with_id[:5]

[([0.0, 0.0, 1.0, 0.0, 0.0], 0),
 ([0.0, 0.125, 0.875, 0.0, 0.0], 0),
 ([0.0, 0.25, 0.75, 0.0, 0.0], 0),
 ([0.0, 0.375, 0.625, 0.0, 0.0], 0),
 ([0.0, 0.5, 0.5, 0.0, 0.0], 0)]

In [147]:
mapping

{'CrFeNi': {'id': 0, 'elements': ['Cr', 'Fe', 'Ni']},
 'CrFeTi': {'id': 1, 'elements': ['Cr', 'Fe', 'Ti']},
 'CrFeV': {'id': 2, 'elements': ['Cr', 'Fe', 'V']},
 'CrNiTi': {'id': 3, 'elements': ['Cr', 'Ni', 'Ti']},
 'CrNiV': {'id': 4, 'elements': ['Cr', 'Ni', 'V']},
 'CrTiV': {'id': 5, 'elements': ['Cr', 'Ti', 'V']},
 'FeNiTi': {'id': 6, 'elements': ['Fe', 'Ni', 'Ti']},
 'FeNiV': {'id': 7, 'elements': ['Fe', 'Ni', 'V']},
 'FeTiV': {'id': 8, 'elements': ['Fe', 'Ti', 'V']},
 'NiTiV': {'id': 9, 'elements': ['Ni', 'Ti', 'V']}}

In [148]:
def reduce_compositions(compositions_with_id, mapping):
    element_order = elementalSpaceComponents
    system_comps_with_id = []
    reduced_compositions = []
    
    for composition, comp_id in compositions_with_id:
        relevant_entry = next((entry for entry in mapping.values() if entry['id'] == comp_id), None)
        if not relevant_entry:
            continue
        
        relevant_indices = [element_order.index(elem) for elem in relevant_entry['elements']]
        reduced_point = [composition[index] for index in relevant_indices]
        system_comps_with_id.append((reduced_point, comp_id))
        reduced_compositions.append(reduced_point)
    
    return system_comps_with_id, reduced_compositions

In [149]:
system_comps_with_id, reduced_compositions = reduce_compositions(compositions_with_id, mapping)
print(reduced_id_list[:5])
print(reduced_compositions[:5])

[([0.0, 0.0, 1.0], 0), ([0.0, 0.125, 0.875], 0), ([0.0, 0.25, 0.75], 0), ([0.0, 0.375, 0.625], 0), ([0.0, 0.5, 0.5], 0)]
[[0.0, 0.0, 1.0], [0.0, 0.125, 0.875], [0.0, 0.25, 0.75], [0.0, 0.375, 0.625], [0.0, 0.5, 0.5]]


In [150]:
from tqdm.contrib.concurrent import process_map

In [151]:
startingNodes = [0, 90, 200, 310] + random.sample(range(len(reduced_compositions)), 11)
print(f"Starting nodes: {startingNodes}")

for startingNode in startingNodes:
    print(f"Starting node: {reduced_compositions[startingNode]}")

Starting nodes: [0, 90, 200, 310, 26, 137, 44, 393, 208, 136, 55, 429, 447, 19, 194]
Starting node: [0.0, 0.0, 1.0]
Starting node: [0.0, 0.0, 1.0]
Starting node: [0.25, 0.375, 0.375]
Starting node: [0.75, 0.125, 0.125]
Starting node: [0.375, 0.25, 0.375]
Starting node: [0.0, 0.25, 0.75]
Starting node: [1.0, 0.0, 0.0]
Starting node: [0.5, 0.375, 0.125]
Starting node: [0.375, 0.5, 0.125]
Starting node: [0.0, 0.125, 0.875]
Starting node: [0.125, 0.125, 0.75]
Starting node: [0.375, 0.0, 0.625]
Starting node: [0.875, 0.0, 0.125]
Starting node: [0.25, 0.25, 0.5]
Starting node: [0.125, 0.625, 0.25]


In [152]:
gridFeasible = [None]*len(reduced_compositions)
queue = startingNodes.copy()
explored = set()
calcCount = 0

In [153]:
# Function to get the correct callable for a given composition
def get_callable(composition):
    for comp, id in compositions_with_id:
        if comp == composition:
            callable_name = id_to_callable.get(id)
            if callable_name is None:
                raise ValueError(f"No callable name found for ID {id}")
            if callable_name in globals():
                return globals()[callable_name]
            else:
                raise NameError(f"Function '{callable_name}' not found in global scope")
    raise ValueError(f"No callable found for composition {composition}")

def process_composition(elP):
    try:
        callable_func = get_callable(elP)
        return callable_func(elP)
    except Exception as e:
        print(f"Error processing composition {elP}: {str(e)}")
        return None


In [154]:
equilibrium_callables

{'equilibrium_callable_CrFeTi_08bbfb9a': <function ammap.callables.multi_system_equilibrium_and_scheil.equilibrium_callable_CrFeTi_08bbfb9a.equilibrium_callable(elP)>,
 'equilibrium_callable_CrTiV_ed4c332b': <function ammap.callables.multi_system_equilibrium_and_scheil.equilibrium_callable_CrTiV_ed4c332b.equilibrium_callable(elP)>,
 'equilibrium_callable_FeTiV_fa95b3ee': <function ammap.callables.multi_system_equilibrium_and_scheil.equilibrium_callable_FeTiV_fa95b3ee.equilibrium_callable(elP)>,
 'equilibrium_callable_NiCrFe_f434a6d9': <function ammap.callables.multi_system_equilibrium_and_scheil.equilibrium_callable_NiCrFe_f434a6d9.equilibrium_callable(elP)>,
 'equilibrium_callable_CrFeV_ab1edb07': <function ammap.callables.multi_system_equilibrium_and_scheil.equilibrium_callable_CrFeV_ab1edb07.equilibrium_callable(elP)>,
 'equilibrium_callable_FeNiV_b9b0384d': <function ammap.callables.multi_system_equilibrium_and_scheil.equilibrium_callable_FeNiV_b9b0384d.equilibrium_callable(elP)>,


In [155]:
id_to_callable

{1: 'equilibrium_callable_CrFeTi_08bbfb9a',
 5: 'equilibrium_callable_CrTiV_ed4c332b',
 8: 'equilibrium_callable_FeTiV_fa95b3ee',
 0: 'equilibrium_callable_NiCrFe_f434a6d9',
 2: 'equilibrium_callable_CrFeV_ab1edb07',
 7: 'equilibrium_callable_FeNiV_b9b0384d',
 6: 'equilibrium_callable_FeNiTi_06a49695',
 3: 'equilibrium_callable_CrNiTi_a9f6f2ff',
 9: 'equilibrium_callable_NiTiV_1d83c99c',
 4: 'equilibrium_callable_NiCrV_b7aba9ab'}

In [156]:
compositions_with_id[:5]

[([0.0, 0.0, 1.0, 0.0, 0.0], 0),
 ([0.0, 0.125, 0.875, 0.0, 0.0], 0),
 ([0.0, 0.25, 0.75, 0.0, 0.0], 0),
 ([0.0, 0.375, 0.625, 0.0, 0.0], 0),
 ([0.0, 0.5, 0.5, 0.0, 0.0], 0)]

In [157]:
type(compositions_with_id)

list

In [158]:
sampled_compositions = compositions_with_id[::50]

# Print the sampled compositions to verify
for composition in sampled_compositions:
    print(composition)

([0.0, 0.0, 1.0, 0.0, 0.0], 0)
([0.0, 0.625, 0.0, 0.375, 0.0], 1)
([0.125, 0.125, 0.0, 0.0, 0.75], 2)
([0.125, 0.0, 0.75, 0.125, 0.0], 3)
([0.25, 0.0, 0.375, 0.0, 0.375], 4)
([0.375, 0.0, 0.0, 0.125, 0.5], 5)
([0.0, 0.5, 0.0, 0.5, 0.0], 6)
([0.0, 0.625, 0.0, 0.0, 0.375], 7)
([0.0, 0.75, 0.0, 0.125, 0.125], 8)


In [159]:
sampled_compositions

[([0.0, 0.0, 1.0, 0.0, 0.0], 0),
 ([0.0, 0.625, 0.0, 0.375, 0.0], 1),
 ([0.125, 0.125, 0.0, 0.0, 0.75], 2),
 ([0.125, 0.0, 0.75, 0.125, 0.0], 3),
 ([0.25, 0.0, 0.375, 0.0, 0.375], 4),
 ([0.375, 0.0, 0.0, 0.125, 0.5], 5),
 ([0.0, 0.5, 0.0, 0.5, 0.0], 6),
 ([0.0, 0.625, 0.0, 0.0, 0.375], 7),
 ([0.0, 0.75, 0.0, 0.125, 0.125], 8)]

In [160]:
print(equilibrium_callables['equilibrium_callable_CrNiTi_a9f6f2ff'](compositions[200]))
compositions[200]

{'Phases': ['LAVES_C15', 'NITI'], 'PhaseFraction': [0.2751953799772505, 0.724804620111501]}


[0.25, 0.0, 0.375, 0.0, 0.375]

In [161]:
from functools import partial

def get_equilibrium_callable(composition, id_to_callable, equilibrium_callables):
    composition_id = composition[1]  # Get the ID from the composition tuple
    #print(f"Composition ID: {composition_id}")
    callable_name = id_to_callable.get(composition_id)
    if callable_name is None:
        raise ValueError(f"No callable found for composition ID {composition_id}")
    callable_func = equilibrium_callables.get(callable_name)
    if callable_func is None:
        raise ValueError(f"No callable function found for name {callable_name}")
    return callable_func

def apply_equilibrium_callable(callable_and_position):
    callable_func, position = callable_and_position
    return callable_func(position)

In [162]:
while len(queue) > 0:
    print(f"Queue: {queue}")
    # Calculate feasibilities of the current queue
    elPositions = [reduced_compositions[i] for i in queue]
    print(elPositions)
    # Create a list of equilibrium callables for each composition
    equilibrium_callables_list = [get_equilibrium_callable(system_comps_with_id[i], id_to_callable, equilibrium_callables) for i in queue]
    
    if len(queue) > 3:
        results = process_map(apply_equilibrium_callable, zip(equilibrium_callables_list, elPositions), max_workers=4)
    else:
        results = [ec(elP) for ec, elP in zip(equilibrium_callables_list, elPositions)]
    
    # Extract only the 'Phases' component from the results
    phases = [result['Phases'] for result in results]
    
    feasibilities = [len(set(p) & set(['FCC_A1', 'BCC_A2', 'HCP_A3', 'B2_BCC','A2_FCC'])) == 0 and p != [] for p in phases]

    calcCount += len(feasibilities)
    explored = explored.union(queue)

    # Create next queue based on neighbors of feasible points
    nextQueue = set()
    nextQueuePlusEquivalent = set()
    for f, i in zip(feasibilities, queue):
        # Explored point
        gridFeasible[i] = f

        # And equivalent explored points based on system stitching
        explored = explored.union(graphNS[i])
        for eq in graphNS[i]:
            gridFeasible[eq] = f

        # Expand to neighbors of the point and equivalent points (only if the node has been feasible)
        if f:
            # Node neighbors in the same subsystem
            for n in graphN[i]:
                if n not in explored and n not in nextQueuePlusEquivalent:
                    nextQueue.add(n)
                    nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])
            # Equivalent nodes neighbors in other subsystems
            for eq in graphNS[i]:
                for n in graphN[eq]:
                    if n not in explored and n not in nextQueuePlusEquivalent:
                        nextQueue.add(n)
                        nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])

    print(f"Calculations done: {calcCount:<5} | Explored points: {len(explored):<5}")
    queue = list(nextQueue)

Queue: [0, 90, 200, 310, 26, 137, 44, 393, 208, 136, 55, 429, 447, 19, 194]
[[0.0, 0.0, 1.0], [0.0, 0.0, 1.0], [0.25, 0.375, 0.375], [0.75, 0.125, 0.125], [0.375, 0.25, 0.375], [0.0, 0.25, 0.75], [1.0, 0.0, 0.0], [0.5, 0.375, 0.125], [0.375, 0.5, 0.125], [0.0, 0.125, 0.875], [0.125, 0.125, 0.75], [0.375, 0.0, 0.625], [0.875, 0.0, 0.125], [0.25, 0.25, 0.5], [0.125, 0.625, 0.25]]


0it [00:00, ?it/s]

Calculations done: 15    | Explored points: 31   
Queue: [387, 388, 392, 394, 397, 398, 422, 423, 46, 47, 430, 306, 307, 435, 309, 54, 311, 312, 313, 56, 62, 63]
[[0.375, 0.375, 0.25], [0.375, 0.5, 0.125], [0.5, 0.25, 0.25], [0.5, 0.5, 0.0], [0.625, 0.25, 0.125], [0.625, 0.375, 0.0], [0.25, 0.0, 0.75], [0.25, 0.125, 0.625], [0.0, 0.125, 0.875], [0.0, 0.25, 0.75], [0.375, 0.125, 0.5], [0.625, 0.125, 0.25], [0.625, 0.25, 0.125], [0.5, 0.0, 0.5], [0.75, 0.0, 0.25], [0.125, 0.0, 0.875], [0.75, 0.25, 0.0], [0.875, 0.0, 0.125], [0.875, 0.125, 0.0], [0.125, 0.25, 0.625], [0.25, 0.0, 0.75], [0.25, 0.125, 0.625]]


0it [00:00, ?it/s]

Calculations done: 37    | Explored points: 65   
Queue: [386, 389, 391, 396, 400, 401, 145, 403, 402, 280, 152, 153, 414, 287, 416, 415, 288, 424, 45, 302, 431, 48, 301, 305, 303, 436, 308, 440, 57, 314, 64, 69, 70, 247, 352, 355, 357, 358, 232, 367, 240, 375, 380, 381, 382]
[[0.375, 0.25, 0.375], [0.375, 0.625, 0.0], [0.5, 0.125, 0.375], [0.625, 0.125, 0.25], [0.75, 0.125, 0.125], [0.75, 0.25, 0.0], [0.125, 0.125, 0.75], [0.875, 0.125, 0.0], [0.875, 0.0, 0.125], [0.125, 0.125, 0.75], [0.25, 0.0, 0.75], [0.25, 0.125, 0.625], [0.125, 0.0, 0.875], [0.25, 0.0, 0.75], [0.125, 0.25, 0.625], [0.125, 0.125, 0.75], [0.25, 0.125, 0.625], [0.25, 0.25, 0.5], [0.0, 0.0, 1.0], [0.5, 0.25, 0.25], [0.375, 0.25, 0.375], [0.0, 0.375, 0.625], [0.5, 0.125, 0.375], [0.625, 0.0, 0.375], [0.5, 0.375, 0.125], [0.5, 0.125, 0.375], [0.625, 0.375, 0.0], [0.625, 0.0, 0.375], [0.125, 0.375, 0.5], [1.0, 0.0, 0.0], [0.25, 0.25, 0.5], [0.375, 0.0, 0.625], [0.375, 0.125, 0.5], [0.25, 0.625, 0.125], [0.625, 0.25, 0.1

...

Calculations done: 248   | Explored points: 303  
Queue: [340, 346, 93, 350, 351]
[[0.375, 0.125, 0.5], [0.5, 0.125, 0.375], [0.0, 0.375, 0.625], [0.625, 0.0, 0.375], [0.625, 0.125, 0.25]]


0it [00:00, ?it/s]

Calculations done: 253   | Explored points: 308  
Queue: [101, 333, 334, 339, 341, 92]
[[0.125, 0.25, 0.625], [0.25, 0.125, 0.625], [0.25, 0.25, 0.5], [0.375, 0.0, 0.625], [0.375, 0.25, 0.375], [0.0, 0.25, 0.75]]


0it [00:00, ?it/s]

Calculations done: 259   | Explored points: 314  
Queue: [332, 325, 326]
[[0.25, 0.0, 0.75], [0.125, 0.125, 0.75], [0.125, 0.25, 0.625]]
Calculations done: 262   | Explored points: 316  
Queue: [317, 318, 327]
[[0.0, 0.25, 0.75], [0.0, 0.375, 0.625], [0.125, 0.375, 0.5]]
Calculations done: 265   | Explored points: 319  
Queue: [316, 319]
[[0.0, 0.125, 0.875], [0.0, 0.5, 0.5]]
Calculations done: 267   | Explored points: 321  
Queue: [320, 328]
[[0.0, 0.625, 0.375], [0.125, 0.5, 0.375]]
Calculations done: 269   | Explored points: 323  
Queue: [329, 321]
[[0.125, 0.625, 0.25], [0.0, 0.75, 0.25]]
Calculations done: 271   | Explored points: 325  


## Exploring Complex Formed by All 3-Component Alloy Subsystems of 7-Component Space (w. Infeasibility Gliding)

In [ ]:
elementalSpaceComponents = ["Ti", "Zr", "Hf", "W", "Nb", "Ta", "Mo"]
attainableSpaceComponents = ["Ti50Zr50", "Hf95Ti5", "NbTaWHf", "Mo80Nb10W10", "TiTa2", "Nb96Mo3W1", "Zr49 Hf1 Mo50"]
attainableSpaceComponentPositions = [
    #Ti, Zr, Hf,  W, Nb, Ta, Mo
    [50, 50,  0,  0,  0,  0,  0], 
    [ 5,  0, 95,  0,  0,  0,  0], 
    [ 0,  0, 25, 25, 25, 25,  0], 
    [ 0,  0,  0, 10, 10,  0, 80],
    [33,  0,  0,  0,  0, 66,  0],
    [ 0,  0,  0,  1, 96,  0,  3],
    [ 0, 49,  1,  0,  0,  0, 50]
]    
ternaries = list(combinations(attainableSpaceComponents, 3))
ternaries_CompPos = list(combinations(attainableSpaceComponentPositions, 3))
ndiv = 8
gridAtt, nList = nimplex.simplex_graph_py(3, ndiv)

for tern, terncp in zip(ternaries, ternaries_CompPos):
    print(f"{str(tern):<40} -> {terncp}")

('Ti50Zr50', 'Hf95Ti5', 'NbTaWHf')       -> ([50, 50, 0, 0, 0, 0, 0], [5, 0, 95, 0, 0, 0, 0], [0, 0, 25, 25, 25, 25, 0])
('Ti50Zr50', 'Hf95Ti5', 'Mo80Nb10W10')   -> ([50, 50, 0, 0, 0, 0, 0], [5, 0, 95, 0, 0, 0, 0], [0, 0, 0, 10, 10, 0, 80])
('Ti50Zr50', 'Hf95Ti5', 'TiTa2')         -> ([50, 50, 0, 0, 0, 0, 0], [5, 0, 95, 0, 0, 0, 0], [33, 0, 0, 0, 0, 66, 0])
('Ti50Zr50', 'Hf95Ti5', 'Nb96Mo3W1')     -> ([50, 50, 0, 0, 0, 0, 0], [5, 0, 95, 0, 0, 0, 0], [0, 0, 0, 1, 96, 0, 3])
('Ti50Zr50', 'Hf95Ti5', 'Zr49 Hf1 Mo50') -> ([50, 50, 0, 0, 0, 0, 0], [5, 0, 95, 0, 0, 0, 0], [0, 49, 1, 0, 0, 0, 50])
('Ti50Zr50', 'NbTaWHf', 'Mo80Nb10W10')   -> ([50, 50, 0, 0, 0, 0, 0], [0, 0, 25, 25, 25, 25, 0], [0, 0, 0, 10, 10, 0, 80])
('Ti50Zr50', 'NbTaWHf', 'TiTa2')         -> ([50, 50, 0, 0, 0, 0, 0], [0, 0, 25, 25, 25, 25, 0], [33, 0, 0, 0, 0, 66, 0])
('Ti50Zr50', 'NbTaWHf', 'Nb96Mo3W1')     -> ([50, 50, 0, 0, 0, 0, 0], [0, 0, 25, 25, 25, 25, 0], [0, 0, 0, 1, 96, 0, 3])
('Ti50Zr50', 'NbTaWHf', 'Zr49 Hf1 Mo5

In [ ]:
nList[1]

[0, 2, 9, 10]

In [ ]:
# Edges list for graph plotting and path finding purposes
edges = []
# Connectivity list within each subsystem
graphN = [[] for i in range(len(gridAtt * len(ternaries)))]
# Connectivity list between subsystems
graphNS = [[] for i in range(len(graphN))]
compositions = []
# Iterate over ternaries
for i, terncp in enumerate(ternaries_CompPos):
    offset = i*len(gridAtt)
    for j in range(len(gridAtt)):
        for n in nList[j]:
            edges.append((j+offset,n+offset))
            graphN[j+offset].append(n+offset)
    gridAttTemp, gridElTemp = nimplex.embeddedpair_simplex_grid_fractional_py(terncp, ndiv)
    compositions += gridElTemp

In [ ]:
type(compositions)

list

In [ ]:
stitchingBinaries = {}

for i, combo1 in enumerate(ternaries):
    for j, combo2 in enumerate(ternaries[i+1:], start=i+1):
        common = set(combo1) & set(combo2)
        if len(common) == 2:
            overlap = tuple(sorted(common))
            if overlap not in stitchingBinaries:
                stitchingBinaries[overlap] = []
            stitchingBinaries[overlap].append((i, j))

for overlap, pairs in stitchingBinaries.items():
    print(f"{overlap}: occurs between ternary {pairs}")

('Hf95Ti5', 'Ti50Zr50'): occurs between ternary [(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]
('NbTaWHf', 'Ti50Zr50'): occurs between ternary [(0, 5), (0, 6), (0, 7), (0, 8), (5, 6), (5, 7), (5, 8), (6, 7), (6, 8), (7, 8)]
('Hf95Ti5', 'NbTaWHf'): occurs between ternary [(0, 15), (0, 16), (0, 17), (0, 18), (15, 16), (15, 17), (15, 18), (16, 17), (16, 18), (17, 18)]
('Mo80Nb10W10', 'Ti50Zr50'): occurs between ternary [(1, 5), (1, 9), (1, 10), (1, 11), (5, 9), (5, 10), (5, 11), (9, 10), (9, 11), (10, 11)]
('Hf95Ti5', 'Mo80Nb10W10'): occurs between ternary [(1, 15), (1, 19), (1, 20), (1, 21), (15, 19), (15, 20), (15, 21), (19, 20), (19, 21), (20, 21)]
('Ti50Zr50', 'TiTa2'): occurs between ternary [(2, 6), (2, 9), (2, 12), (2, 13), (6, 9), (6, 12), (6, 13), (9, 12), (9, 13), (12, 13)]
('Hf95Ti5', 'TiTa2'): occurs between ternary [(2, 16), (2, 19), (2, 22), (2, 23), (16, 19), (16, 22), (16, 23), (19, 22), (19, 23), (22, 23)]
('Nb96Mo3W1', 'Ti50Zr50'): occur

In [ ]:
for stitchingBinary, ternaryPairList in stitchingBinaries.items():
    for ternaryPair in ternaryPairList:
        ternary1, ternary2 = ternaryPair[0], ternaryPair[1]
        stitching1 = stitching.findStitchingPoints_py(
            3, ndiv, 
            components=ternaries[ternary1],
            offset=ternary1*len(gridAtt)
            )["-".join(stitchingBinary)]
        stitching2 = stitching.findStitchingPoints_py(
            3, ndiv, 
            components=ternaries[ternary2],
            offset=ternary2*len(gridAtt)
            )["-".join(stitchingBinary)]
        print(f"Stitching {ternary1} and {ternary2} at {stitchingBinary} from {stitching1} to {stitching2}")
        for i, j in zip(stitching1, stitching2):
            edges.append((i, j))
            graphNS[i].append(j)

Stitching 0 and 1 at ('Hf95Ti5', 'Ti50Zr50') from [8, 16, 23, 29, 34, 38, 41, 43, 44] to [53, 61, 68, 74, 79, 83, 86, 88, 89]
Stitching 0 and 2 at ('Hf95Ti5', 'Ti50Zr50') from [8, 16, 23, 29, 34, 38, 41, 43, 44] to [98, 106, 113, 119, 124, 128, 131, 133, 134]
Stitching 0 and 3 at ('Hf95Ti5', 'Ti50Zr50') from [8, 16, 23, 29, 34, 38, 41, 43, 44] to [143, 151, 158, 164, 169, 173, 176, 178, 179]
Stitching 0 and 4 at ('Hf95Ti5', 'Ti50Zr50') from [8, 16, 23, 29, 34, 38, 41, 43, 44] to [188, 196, 203, 209, 214, 218, 221, 223, 224]
Stitching 1 and 2 at ('Hf95Ti5', 'Ti50Zr50') from [53, 61, 68, 74, 79, 83, 86, 88, 89] to [98, 106, 113, 119, 124, 128, 131, 133, 134]
Stitching 1 and 3 at ('Hf95Ti5', 'Ti50Zr50') from [53, 61, 68, 74, 79, 83, 86, 88, 89] to [143, 151, 158, 164, 169, 173, 176, 178, 179]
Stitching 1 and 4 at ('Hf95Ti5', 'Ti50Zr50') from [53, 61, 68, 74, 79, 83, 86, 88, 89] to [188, 196, 203, 209, 214, 218, 221, 223, 224]
Stitching 2 and 3 at ('Hf95Ti5', 'Ti50Zr50') from [98, 106, 113

In [ ]:
len(edges)

9450

In [ ]:
# Plot is quite interesting, being a much more complex version of the last one, but it has too many points for a small Codespace VM to display smootly. You can remove `#` and 
# plotGraph(edges)
clear_output()

In [ ]:
from pycalphad import Database
dbf = Database("CrHfMoNbTaTiVWZr_9element_Feb2023.tdb")
phases = list(set(dbf.phases.keys()))
print(elementalSpaceComponents)
print(f'Loaded TDB file with phases considered: {phases}')

['Ti', 'Zr', 'Hf', 'W', 'Nb', 'Ta', 'Mo']
Loaded TDB file with phases considered: ['LAVES_C14', 'LAVES_C36', 'LIQUID', 'BCC_A2', 'HCP_A3', 'FCC_A1', 'LAVES_C15']


In [ ]:
from myPycalphadCallable import equilibrium_callable

In [ ]:
print(compositions[0])
equilibrium_callable(compositions[0])

[0.0, 0.0, 0.25, 0.25, 0.25, 0.25, 0.0]


['HCP_A3', 'BCC_A2']

In [ ]:
startingNodes = [0, 90, 1527, 1538] + random.sample(range(len(compositions)), 11)
print(f"Starting nodes: {startingNodes}")

for startingNode in startingNodes:
    print(f"Starting node: {compositions[startingNode]}")

Starting nodes: [0, 90, 1527, 1538, 335, 3, 893, 179, 1223, 773, 143, 13, 646, 1499, 918]
Starting node: [0.0, 0.0, 0.25, 0.25, 0.25, 0.25, 0.0]
Starting node: [0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.6666666666666666, 0.0]
Starting node: [0.0, 0.06125, 0.00125, 0.08750000000000001, 0.08750000000000001, 0.0, 0.7625000000000001]
Starting node: [0.0, 0.0, 0.0, 0.01, 0.96, 0.0, 0.03]
Starting node: [0.125, 0.125, 0.09375, 0.0975, 0.45375, 0.09375, 0.01125]
Starting node: [0.018750000000000003, 0.0, 0.5125, 0.15625, 0.15625, 0.15625, 0.0]
Starting node: [0.03125, 0.0, 0.59375, 0.037500000000000006, 0.037500000000000006, 0.0, 0.30000000000000004]
Starting node: [0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0]
Starting node: [0.0, 0.0, 0.0, 0.1, 0.1, 0.0, 0.8]
Starting node: [0.0, 0.0, 0.25, 0.25, 0.25, 0.25, 0.0]
Starting node: [0.05, 0.0, 0.95, 0.0, 0.0, 0.0, 0.0]
Starting node: [0.0875, 0.0625, 0.56875, 0.09375, 0.09375, 0.09375, 0.0]
Starting node: [0.0625, 0.0625, 0.0, 0.00875, 0.84, 0.0, 0.02625]

In [ ]:
from tqdm.contrib.concurrent import process_map

In [ ]:
gridFeasible = [None]*len(compositions)
queue = startingNodes.copy()
explored = set()
calcCount = 0

In [ ]:
while len(queue)>0:
    print(f"Queue: {queue}")
    # Calculate feasibilities of the current queue
    elPositions = [compositions[i] for i in queue]
    if len(queue)>3:
        phases = process_map(equilibrium_callable, elPositions, max_workers=4)
    else:
        phases = [equilibrium_callable(elP) for elP in elPositions]
    feasibilities = [len(set(p) & set(['LAVES_C15', 'LAVES_C36', 'LAVES_C14', 'LIQUID']))==0 and p!=[] for p in phases]

    calcCount += len(feasibilities)
    explored = explored.union(queue)

    # Create next queue based on neighbors of feasible points
    nextQueue = set()
    nextQueuePlusEquivalent = set()
    for f, i in zip(feasibilities, queue):
        # Explored point
        gridFeasible[i] = f

        # And equivalent explored points based on system stitching
        explored = explored.union(graphNS[i])
        for eq in graphNS[i]:
            gridFeasible[eq] = f

        # Expand to neighbors of the point and equivalent points (only if the node has been feasible)
        if f:
            # Node neighbors in the same subsystem
            for n in graphN[i]:
                if n not in explored and n not in nextQueuePlusEquivalent:
                    nextQueue.add(n)
                    nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])
            # Equivalent nodes neighbors in other subsystems
            for eq in graphNS[i]:
                for n in graphN[eq]:
                    if n not in explored and n not in nextQueuePlusEquivalent:
                        nextQueue.add(n)
                        nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])

    print(f"Calculations done: {calcCount:<5} | Explored points: {len(explored):<5}")
    queue = list(nextQueue)

Queue: [0, 90, 1527, 1538, 335, 3, 893, 179, 1223, 773, 143, 13, 646, 1499, 918]


  0%|          | 0/15 [00:00<?, ?it/s]

Calculations done: 15    | Explored points: 51   
Queue: [1, 1537, 2, 4, 5, 645, 9, 1546, 11, 12, 1033, 142, 271, 14, 653, 911, 787, 20, 277, 406, 151, 21, 793, 917, 919, 924, 925, 547, 808, 943, 177, 178, 697, 187, 703, 832, 322, 838, 327, 328, 583, 1222, 1483, 334, 1231, 592, 336, 1490, 1491, 341, 342, 1498, 91, 1500, 222, 1506, 99, 1507, 910, 742, 358, 232, 748, 493, 367, 1528, 637, 638]


  0%|          | 0/68 [00:00<?, ?it/s]

Calculations done: 83    | Explored points: 216  
Queue: [1536, 6, 1031, 1030, 1545, 10, 1032, 1543, 1544, 15, 17, 1553, 18, 19, 22, 535, 26, 27, 28, 1050, 538, 546, 1570, 1574, 555, 1075, 1077, 1079, 1088, 580, 581, 1095, 1102, 591, 1108, 600, 92, 1120, 100, 1125, 1126, 107, 1135, 625, 627, 629, 636, 644, 652, 141, 1165, 1169, 659, 150, 158, 670, 672, 682, 174, 175, 176, 689, 690, 696, 698, 1210, 195, 708, 712, 713, 1230, 1238, 727, 729, 730, 219, 220, 734, 735, 741, 231, 1255, 747, 240, 753, 1267, 757, 1275, 772, 779, 780, 272, 786, 1300, 276, 279, 280, 792, 285, 798, 1312, 802, 805, 1320, 817, 824, 825, 318, 831, 319, 1345, 321, 320, 837, 326, 329, 330, 843, 333, 847, 337, 850, 852, 340, 854, 343, 1365, 1366, 346, 347, 348, 865, 355, 356, 1390, 366, 375, 1410, 901, 902, 903, 909, 912, 400, 403, 404, 407, 920, 926, 415, 930, 931, 940, 1455, 1480, 1481, 1489, 1492, 1497, 1501, 1505, 1000, 1512, 490, 1513, 1005, 1008, 1525, 1014, 1015, 1526]


...

Calculations done: 890   | Explored points: 1218 
Queue: [448, 961, 421, 134, 135, 871, 455, 269, 144, 464, 1141, 952, 443, 1148, 152, 413, 159]


  0%|          | 0/17 [00:00<?, ?it/s]

Calculations done: 907   | Explored points: 1238 
Queue: [456, 457, 1133, 502]


  0%|          | 0/4 [00:00<?, ?it/s]

Calculations done: 911   | Explored points: 1241 
Queue: [465]
Calculations done: 912   | Explored points: 1242 


You should see around 930-950 calculations done and around 1250 points explored (thanks to equivalence of stitching points).

In [ ]:
feasibleCount = 0
for f in gridFeasible:
    if f:
        feasibleCount += 1
print(f"Feasible points: {feasibleCount} out of {len(gridFeasible)}")

Feasible points: 1039 out of 1575


Out of the explored points, around 1040 should be feasible (i.e., satisfy the equilibrium constraint).

In [ ]:
# You can double check the feasibility of the points by running the following code, it is skipped here to save time during the tutorial

#gridPhasesComplete = process_map(equilibrium_callable, compositions)

#gridFeasibleComplete = [len(set(p) & set(['LAVES_C15', 'LAVES_C36', 'LAVES_C14', 'LIQUID']))==0 and p!=[] for p in gridPhasesComplete]
#print(sum(gridFeasibleComplete))

1575

## Adding More Constraints

In [ ]:
import pqam_rmsadtandoc2023

def c2rmsad(point):
    formula = ' '.join([f'{c}{p}' for c, p in zip(elementalSpaceComponents, point)])
    return pqam_rmsadtandoc2023.predict(formula)

In [ ]:
print(compositions[0])
c2rmsad(compositions[0])

[0.0, 0.0, 0.25, 0.25, 0.25, 0.25, 0.0]


0.11816658059128823

In [ ]:
avgRMSAD = sum([c2rmsad(p) for p in compositions])/len(compositions)
print(f"Average RMSAD: {avgRMSAD}")

Average RMSAD: 0.16141834794438295


In [ ]:
startingNodes = [0, 90, 742, 799, 1048, 1070, 1538] + random.sample(range(len(compositions)), 13)

for startingNode in startingNodes:
    print(f"Starting node ({startingNode}): {compositions[startingNode]}")

Starting node (0): [0.0, 0.0, 0.25, 0.25, 0.25, 0.25, 0.0]
Starting node (90): [0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.6666666666666666, 0.0]
Starting node (742): [0.05416666666666667, 0.0, 0.39375, 0.15625, 0.15625, 0.23958333333333331, 0.0]
Starting node (799): [0.025, 0.0, 0.6, 0.125, 0.125, 0.125, 0.0]
Starting node (1048): [0.17291666666666666, 0.18375, 0.1225, 0.0, 0.0, 0.3333333333333333, 0.1875]
Starting node (1070): [0.03125, 0.18375, 0.5975, 0.0, 0.0, 0.0, 0.1875]
Starting node (1538): [0.0, 0.0, 0.0, 0.01, 0.96, 0.0, 0.03]
Starting node (680): [0.0, 0.0, 0.15625, 0.19375, 0.19375, 0.15625, 0.30000000000000004]
Starting node (697): [0.0125, 0.0, 0.39375, 0.16875, 0.16875, 0.15625, 0.1]
Starting node (106): [0.10625, 0.0625, 0.8312499999999999, 0.0, 0.0, 0.0, 0.0]
Starting node (326): [0.0625, 0.0625, 0.0625, 0.06875, 0.6625, 0.0625, 0.01875]
Starting node (276): [0.08333333333333333, 0.0, 0.1875, 0.1875, 0.1875, 0.35416666666666663, 0.0]
Starting node (690): [0.00625, 0.0,

In [ ]:
gridFeasible = [None]*len(compositions)
queue = startingNodes.copy()
explored = set()
calcCount = 0

minRMSAD = 0.2
print(f"Minimum RMSAD: {minRMSAD}")

Minimum RMSAD: 0.2


In [ ]:
while len(queue)>0:
    print(f"Queue: {queue}")
    # Skip all points with RMSAD below the minimum (inexpensive) by removing them from the queue, marking them as infeasible, and adding them to explored
    for i in queue:
        if c2rmsad(compositions[i]) < minRMSAD:
            for eq in [i] + graphNS[i]:
                gridFeasible[eq] = False
            explored = explored.union([i] + graphNS[i])
    queue = [i for i in queue if gridFeasible[i] is None]

    # Calculate CALPHAD feasibilities (expensive) of the current queue
    elPositions = [compositions[i] for i in queue]
    if len(queue)>3:
        phases = process_map(equilibrium_callable, elPositions, max_workers=4)
    else:
        phases = [equilibrium_callable(elP) for elP in elPositions]
    feasibilities = [len(set(p) & set(['LAVES_C15', 'LAVES_C36', 'LAVES_C14', 'LIQUID']))==0 and p!=[] for p in phases]

    calcCount += len(feasibilities)
    explored = explored.union(queue)

    # Create next queue based on neighbors of feasible points
    nextQueue = set()
    nextQueuePlusEquivalent = set()
    for f, i in zip(feasibilities, queue):
        # Explored point
        gridFeasible[i] = f

        # And equivalent explored points based on system stitching
        explored = explored.union(graphNS[i])
        for eq in graphNS[i]:
            gridFeasible[eq] = f

        # Expand to neighbors of the point and equivalent points (only if the node has been feasible)
        if f:
            # Node neighbors in the same subsystem
            for n in graphN[i]:
                if n not in explored and n not in nextQueuePlusEquivalent:
                    nextQueue.add(n)
                    nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])
            # Equivalent nodes neighbors in other subsystems
            for eq in graphNS[i]:
                for n in graphN[eq]:
                    if n not in explored and n not in nextQueuePlusEquivalent:
                        nextQueue.add(n)
                        nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])

    print(f"Calculations done: {calcCount:<5} | Explored points: {len(explored):<5}")
    queue = list(nextQueue)

Queue: [0, 90, 742, 799, 1048, 1070, 1538, 680, 697, 106, 326, 276, 690, 1148, 683, 1437, 502, 335, 3, 893]
Calculations done: 3     | Explored points: 62   
Queue: [97, 98, 803, 195, 838, 105, 843, 142, 113, 150, 793, 187, 798]


  0%|          | 0/12 [00:00<?, ?it/s]

Calculations done: 15    | Explored points: 96   
Queue: [898, 1030, 1032, 141, 149, 792, 157, 797, 802, 805, 806, 940, 944, 1075, 1077, 831, 832, 837, 842, 202, 847, 96, 1120, 104, 112, 119, 760, 763, 764, 895]


  0%|          | 0/24 [00:00<?, ?it/s]

Calculations done: 39    | Explored points: 140  
Queue: [1026, 1027, 1029, 1031, 140, 148, 791, 796, 156, 801, 163, 164, 936, 808, 809, 1071, 1072, 1074, 208, 850, 1116, 1117, 95, 103, 111, 756, 757, 118, 761, 891, 124]


...

Calculations done: 188   | Explored points: 440  
Queue: [391, 392, 393, 395, 398, 656, 657, 661, 289, 290, 297, 303, 308, 438, 443, 341, 342, 348, 605, 611, 483, 616]


  0%|          | 0/12 [00:00<?, ?it/s]

Calculations done: 200   | Explored points: 468  
Queue: [386, 387, 388, 291, 610, 615, 394, 298, 655, 304, 660, 343, 604]


  0%|          | 0/9 [00:00<?, ?it/s]

Calculations done: 209   | Explored points: 482  
Queue: [389, 299, 380, 381, 382]
Calculations done: 209   | Explored points: 488  


In [ ]:
sum([f for f in gridFeasible if f])

286

In [ ]:
len(gridFeasible)

1575